In [1]:
# %%capture
# Capture because braindecode module prints a lot :S
from pyFiles.training import *
from pyFiles.models import *
from pyFiles.dataprep_down import *
from pyFiles.dataprep2_down import *
#
import torch
from torch import nn
from torch.nn import functional as F
#
from pyFiles.losses import InfoNceLoss, VICReg, BarlowTwins

c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\docstring_inheritance\docstring_inheritors\bases\inheritor.py:329: DocstringInheritanceWarning: File c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\braindecode\models\eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_outputs' is missing.
  self._warn(
c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\docstring_inheritance\docstring_inheritors\bases\inheritor.py:329: DocstringInheritanceWarning: File c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\braindecode\models\eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_chans' is missing.
  self._warn(
c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\docstring_inheritance\docstring_inheritors\bases\inheritor.py:329: DocstringInheritanceWarning: File c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\braindecode\models\eegconformer.py:88: in EEGConformer.

In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [3]:
#PREP DENUEVO
# prep("C:\\dataverse_files", mode = "per_channel", save = True, save_dir = "data_down")

In [4]:
# spliter = DFSpliter(save = True, mode= "down_ch")
# spliter("down_prep_channels.csv",".")

In [5]:
transforms = Augmentations(5, False, AUGMENTATIONS)

In [6]:
p_size = int(19) #int(1216/2) # Max = 19*64 = 1216
class CustomEEGDataset(Dataset):
    def __init__(self, csv_file , root_dir , transform = None, ):

        try:
            self.loc_df = pd.read_csv(os.path.join(root_dir,csv_file)).drop(labels="Unnamed: 0", axis = 1)
        except:
            self.loc_df = pd.read_csv(os.path.join(root_dir,csv_file))

        self.transform = transform
        self.root_dir = root_dir

    def __len__(self,):
        return len(self.loc_df)//p_size
    

    def __getitem__(self, idx):
        exam_shape = (p_size, 1, 1000)
        # print(val_shape)
            # return ch, diagnostic
        new_exam = torch.ones(exam_shape)
        # for i in range(len(self.loc_df)):
        for j in range(p_size):
            eeg_file = os.path.join(self.root_dir,
                            self.loc_df.iloc[idx*p_size + j, 2])
            ch, diagnostic = torch.load(eeg_file)
        
            if self.transform is not None:
                ch = self.transform(ch)
            new_exam[j] = ch
        
        return new_exam, diagnostic

In [7]:
train_dataset = CustomEEGDataset("down_ch_train_feats.csv", root_dir = "", transform = transforms)
val_dataset = CustomEEGDataset("down_ch_val_feats.csv", root_dir = "")

In [14]:
# trainloader = DataLoader(train_dataset, batch_size = 1, shuffle = True, num_workers = 0)
trainloader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 0)
valloader = DataLoader(val_dataset, batch_size = 12, shuffle = False, num_workers = 0)

In [15]:
model = SleepStagerSingle(n_chans = 1, pretext = False, p_size= p_size)
enc, fc = torch.load("params\singles\VR\\attempt3\\bestEncoderParamssingle_vr.pt")
model.encoder.load_state_dict(enc)

criterion = nn.CrossEntropyLoss()

for param in model.encoder.feature_extractor[0].parameters():
    param.requires_grad = False

optimizer = torch.optim.Adam(
    [
        {"params": model.encoder.parameters(), "lr": 3e-8},
        {"params": model.final.parameters(), "lr": 3e-5},
    ],
    lr=5e-4,
) 
downtrain(model, 30, trainloader, valloader, criterion, optimizer, state = None, name = "single_VR")

Epoch nro 1/30
Iter: 1/44, Loss:0.8381750583648682
Iter: 2/44, Loss:0.7444983720779419
Iter: 3/44, Loss:0.7274100184440613
Iter: 4/44, Loss:0.6873390674591064
Iter: 5/44, Loss:0.6513490676879883
Iter: 6/44, Loss:0.6753884553909302
Iter: 7/44, Loss:0.8369171619415283
Iter: 8/44, Loss:0.7540644407272339
Iter: 9/44, Loss:0.7431977391242981
Iter: 10/44, Loss:0.8375343084335327
Iter: 11/44, Loss:0.6552203893661499
Iter: 12/44, Loss:0.8227384090423584
Iter: 13/44, Loss:0.7594208121299744
Iter: 14/44, Loss:0.7238601446151733
Iter: 15/44, Loss:0.8145702481269836
Iter: 16/44, Loss:0.768802285194397
Iter: 17/44, Loss:0.7609533667564392
Iter: 18/44, Loss:0.8054344058036804
Iter: 19/44, Loss:0.7672271132469177
Iter: 20/44, Loss:0.7849051356315613
Iter: 21/44, Loss:0.7339490056037903
Iter: 22/44, Loss:0.8984308242797852
Iter: 23/44, Loss:0.7405956387519836
Iter: 24/44, Loss:0.8055883646011353
Iter: 25/44, Loss:0.7546695470809937
Iter: 26/44, Loss:0.7747538685798645
Iter: 27/44, Loss:0.8020839691162

([[0.7499694322997873,
   0.7402629107236862,
   0.7253787571733649,
   0.7209010828625072,
   0.7111134136264975,
   0.7061011953787371,
   0.7033426490696993,
   0.6984433802691373,
   0.6976038149812005,
   0.6929260763255033,
   0.689381168647246,
   0.6887198158285834,
   0.6840820461511612,
   0.6889753138477152,
   0.6851674128662456,
   0.6865876628593965,
   0.681595119563016,
   0.6841388696973975,
   0.6784301671114835,
   0.6776720366694711,
   0.678160774436864,
   0.6721179702065208,
   0.6714958833022551,
   0.6710066985000264,
   0.6669313121925701,
   0.6682717488570646,
   0.6683495491743088,
   0.6675542796199972,
   0.6680317534641786,
   0.665116467259147],
  [0.7395710991695523,
   0.7249318463727832,
   0.7137776231393218,
   0.7054046522825956,
   0.6990240337327123,
   0.6941863931715488,
   0.6902186721563339,
   0.6878983583301306,
   0.6858486849814653,
   0.6847527604550123,
   0.6831875517964363,
   0.682763921096921,
   0.6821472328156233,
   0.6817972864